"(잔업이다)" 라는 tag를 추가로 조사하거나 공부해야할 부분에 붙여 두었다. 공부하자.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
#습관적으로 import해야하는 torch package들이다.

3번째로 구현한 class. 정의 순서 때문에, 이렇게 역순으로 작성을 한다. 그보다 논문을 읽은지 너무 오래되어서, ResNet의 downsampling부분에 대한 헷갈림이 있는 것 같다. -> 확인해보자.

In [ ]:
class IdentityPadding(nn.Module):
  def __init__(self, in_channels, out_channels, stride):
    super(IdentityPadding,self).__init__()

    self.pooling = nn.MaxPool2d(1,stride=stride)
    #이게 호출될때는 stride가 2거나 그럴테니, 이건 down sampling을 수행할 것.
    self.add_channels = out_channels - in_channels
  
  def forward(self, x):
    out = F.pad(x, (0,0,0,0,0,self.add_channels))
    #이거는 feature map의 마지막 축에대해서는 (0,0) padding, 마지막에서 2번째 축에 대해서는 (0,0) padding, 마지막에서 3번째 축에 대해서는 (0,self.add_channels)만큼 padding을 하라는 의미이다.
    #최종적으로 x는 channels축에 대해서 out_channels - in_channels만큼 (아마 2배로) padding될 것이고 zero padding으로 처리될 것이다.

    #근데 이러면 끝에 0들이 붙는건데 학습이 되나? (잔업이다)

    out = self.pooling(out)

    return out



2번째로 구현한 class. 정의 순서 때문에, 이렇게 역순으로 작성을 한다.

In [ ]:
class ResidualBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride=1, down_sample = False):
    #앞서 ResNet에서 정의한 block에 들어가는 argument들로 초기화가 되는 것임
    super(ResidualBlock, self).__init__()
    #항상 그렇든 상속받은거라 실행하고
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU(inplace = True)
    #이번엔 block의 처음을 구성할 conv1을 정의하고 따라서 bn1을 정의했다. conv1은 downsampling을 해야하니, 위와같이 따로 정의해줘야 한다.
    #ResNet이기에 모두 동일한 3x3 filter로 convolution을 취한다.

    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(out_channels)
    #이건 bn1이랑 같아서 정의 안해도 되는데, 그냥 흐름상 정의한 것 같다. 따라하자.
    self.stride = stride
    #내부 변수에 저장

    if down_sample:
      self.down_sample = IdentityPadding(in_channels,out_channels, stride)
      #만약 down_sample을 해야하는 첫 레이어라면 Identity padding을 가해서 크기를 변경하는 것.
      #만약 input argument down_sample이 True면, self.down_sample을 layer에 추가하는 듯.
      #이는 Downsample이 일어나는 첫 layer에서 shortcut 구현에 있어 추가해준 것
      #논문에선 다양한 방식을 시도해봤었는데, 조사해보자. (잔업이다)
    else:
      self.down_sample=None

  def forward(self,x):
    shortcut = x

    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    #첫 layer를 정의함.
    #첫 호출만 아니면 in_channels == out_channels이기에 그대로 써도 문제가 없음. conv1==conv2로 작동할 것임.

    out = self.conv2(out)
    out = self.bn2(out)

    if self.down_sample is not None:
      shortcut = self. down_sample(x)
    #처음에 크기가 변하는 layer에 대해서 shortcut 처리해주기.

    out += shortcut
    #shortcut을 더해준다.
    out = self.relu(out)
    #이렇게 shortcut을 더하고 relu를 가하는 것은 relu를 먼저가하면 양의 부분만 남아서, shortcut의 의미가 사라지기 때문이라네.

    return out


아래는 ResNet을 정의하는 class이다. 
github.com/dnddnjs/pytorch-cifar10/blob/enas/resnet/의 파일을 참고했다.

가장 먼저 작성한 블락이나, 의존성 때문에 위에 추가적으로 코드를 작성했다.

In [ ]:
class ResNet(nn.Module):
  #class 형태의 module은 nn.Module을 항상 상속받아야 한다.
  def __init__(self, num_layers, block, num_classes = 10):
    super(ResNet, self).__init__()
    #nn.Module.__init__()을 실행해 instance를 만들어야함.
    #super는 부모 class의 initalizer를 불러와, 그안의 정보를 사용하고 싶을 때 사용하는 것임. 
    #이렇게 호출하지 않으면, 덮어씌워지기 때문에 문제의 소지가 큼.
    self.num_layers = num_layers 
    #입력 argument를 복사하기.
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False)
    #입력 이미지에 대해 3x3 Conv(kernel_size가 3)를 가하는데, in_channels가 3인 이유는 RGB가 들어오기 떄문이다. 
    #3x32x32가 들어와서 16x32x32의 feature map이 나가게 된다. 즉 3x3x3짜리 filter가 16set 나오는 것.

    self.bn1 = nn.BatchNorm2d(16)
    #depth 16짜리 batch normalization을 추가.
    self.relu=nn.ReLU(inplace=True)
    #inplace를 True로 설정하는 것은 input에서 바로 output으로 바꾼다는 뜻인데, 추가적인 output을 allocation하지 않으니 memory 절약이 가능하다.
    #다만, 주의해서 사용해야 한다.


    self.layers_2n = self.get_layers(block,16,16,stride=1)
    #feature map size 32x32x16 인 2n개의 layer를 구현할 것
    self.layers_4n = self.get_layers(block,16,32,stride=2)
    #feature map size 16x16x32인 2n~4n layer를 구현할 것
    self.layers_6n = self.get_layers(block,32,64,stride=2)
    #feature map이 8x8x64인 4n~6n layer를 구현할 것

    
    #여기서부턴 출력 layer이다.

    self.avg_pool = nn.AvgPool2d(8,stride = 1)
    #마지막에 average pooling을 8x8 featuremap에 대해서 가할 것이다.(kernel size가 8이라 사실상 전체의 average를 구하는 것) -> 논문 확인 필요 (잔업이다)
    self.fc_out= nn.Linear(64, num_classes)
    # 64depth의 final output을 num_classes로 이어지는 FCL을 만든다.

    for m in self.modules():
      # 이 class에 있는 module에 대해서, 이들이 해당하는 instance일 경우 초기화를 해주는 역할이다.
      if isinstance(m,nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode = 'fan_out' , nonlinearity='relu')
        #He initialization이라고 한다. normal ditribution을 변형한 거 같은데, 자세히는 추가 조사를 해보자 (잔업이다)
        #fan_in을 선택하면 foward pass에서 weight var의 크기를 유지하고, fan_out을 선택하면, backward pass에서 크기를 유지한다 하는데, initializer가 어떻게 그런거를 관여하지 (잔업이다)
      elif isinstance(m,nn.BatchNorm2d):
        nn.init.constant_(m.weight,1)
        # 1인 상수로 weight를 초기화
        nn.init.constant_(m.bias,0)
        # bais는 0으로 초기화
  
  def get_layers(self, block, in_channels, out_channels, stride):
    if stride ==2:
      down_sample = True
    else:
      down_sample = False
    #stride에 따라서 출력 크기가 바뀌는데, 그 여부를 down_sample에 저장해둠.

    layers_list = nn.ModuleList([block(in_channels, out_channels, stride, down_sample)])
    #입력받은 block을 ModuleList에서 찾아 실행한다.
    #위에서 정의된 부분을 사용할 것이다.

    for _ in range(self.num_layers-1):
      #layer의 갯수만큼 만들어야지.
      layers_list.append(block(out_channels, out_channels))

    return nn.Sequential(*layers_list)
    #근데 *이 의미하는게 뭐지? 일단 layer_list라는 block들의 덩어리를 return하겠다는 건 이해가 되는데. (잔업이다)


  def forward(self,x):
    #forward propagation을 진행하는 부분이다.
    #항상 쓰이며, 여기서 Model의 구조를 위에 정의한 Module들을 가지고 하는 것

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    #첫 입력 신호 차원 바꾸는 conv layer

    x=self.layers_2n(x)
    x=self.layers_4n(x)
    x=self.layers_6n(x)
    #Resnet block들을 이어주고

    x=self.avg_pool(x)
    x=x.view(x.size(0),-1)
    x=self.fc_out(x)
    #최종 출력 layer를 정의한다.

    return x


최종적으로 아래의 함수로 위의 class들을 호출해 ResNet을 정의한다.

최종 레이어는 6n+2개로 구성될 것이다.

In [ ]:
def resnet(n):
  block = ResidualBlock

  model = ResNet(n,block)
  return model

여기까지가 Model의 정의였고, 이제는 training 하는 부분을 만들어야 한다.

skeleton code와 위에 구현된 code를 바탕으로 하나씩 옮겨 적으며 정리를 해보았다.


In [ ]:
%pip install tensorboardX

In [ ]:
import os
from torch.optim import lr_scheduler
#epoch에 따라서 learning rate를 조절해주는 함수이다.
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
#학습에 쓸 CIFAR10데이터 셋과 preprocess를 위한 transform 패키지를 얻는다.

import argparse
# 이건 파일 실행시 인자값을 받는 용도로 정의되는 것임 (잔업이다)
from tensorboardX import SummaryWriter
#좋은 visualizer

이 부분은 parser로 python이 호출되어서 작동할떄, 요구받는 인자이다. 
Python에서만 적용되는 거라 처음보고 익숙하지가 않다... ㅠ

(잔업이다)


notebook에서 구현하다 보니, 작동에 문제가 있는 것 같다. 
일단은 주석처리 해두자.

In [ ]:
'''

parser = argparse.ArgumentParser(description='cifar10 classification models')
#인자값을 받는 instance를 형성한다.

parser.add_argument('--lr',default=0.1, help='')
parser.add_argument('--resume',default= None, help='')
parser.add_argument('--batch_size',default = 128, help='')
parser.add_argument('--batch_size_test',default = 100, help='')
parser.add_argument('--num_worker',default=4,help='')
parser.add_argument('--logdir',type = str, default='logs',help='')
#인자값들을 입력받는 설정을 보여준다.
#원래 trainer 초반에 정의해두는 hyperparameter들을 여기의 default로 설정을 함으로 정의하고 있다.

args = parser.parse_args()
#입력받은 인자값을 args에다가 저장한다.
'''
class Arg:
  def __init__(self):
    self.lr = 0.1
    self.resume = None
    self.batch_size = 128
    self.batch_size_test = 100
    self.num_worker = 4
    self.logdir = 'logs'

args = Arg()


이 부분은 데이터를 받아오고, preprocessing을 가하는 부분이다.

In [ ]:
%mkdir ./data


mkdir: cannot create directory ‘./data’: File exists


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#CUDA가 사용 가능하면 cuda를 쓰고, 그게 아니면 cpu를 사용하자.

print('preprocessing data')

transforms_train = transforms.Compose(
    [
     transforms.RandomCrop(32,padding=4),
     #32x32로 padding 4까지 허용하고 Random Crop
     transforms.RandomHorizontalFlip(),
     #Random하게 Flip
     transforms.ToTensor(),
     transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010)),
     #이건 CIFAR10의 channel별 평균관 std-deviation이다.
     #이거로 standardization을 하는 것
    ]
)
transforms_test = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010)),
    ]
)
#전처리 + 데이터 늘리기를 수행하는 함수? instance?를 정의

dataset_train = CIFAR10(root='./data', train=True, download=True, transform=transforms_train)
dataset_test = CIFAR10(root='./data',train=False ,download=True,transform= transforms_test)
#./data에 CIFAR10을 다운로드 한다

train_loader = DataLoader(dataset_train, batch_size=args.batch_size, shuffle = True, num_workers = args.num_worker)
test_loader = DataLoader(dataset_test,batch_size=args.batch_size_test, shuffle = False, num_workers= args.num_worker)
#DataLoader로 파일들을 불러온다.

preprocessing data
Files already downloaded and verified
Files already downloaded and verified


이제는 Model과 세부 사항을 조절할 차례

In [ ]:
print('Making model')

net = resnet(5)
net = net.to(device)
#설정한 device에서 작동하게 설정

num_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
# 총 parameter의 갯수를 구한다.
# net이 nn.Module을 상속했기에, parameters()나 parameters.requires_grad 같은게 작동함
print('The number of parameters of model is', num_params)
print(net)

if args.resume is not None:
  checkpoint = torch.load('./save_model'+args.resume)
  net.load_state_dict(checkpoint['net'])
#만약 resume하는 경우엔, ./save_model에서 불러와 net에다가 집어 넣는 역할이다.


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr= args.lr, momentum = 0.9, weight_decay = 1e-4)
#cross entropy를 loss로 사용하고, SGD를 optimizer로 설정했다
#Momentum은 0.9, weight_decay는 0.0001로 설정헀는데, 이거 앞에 상수로 빼주면 훨씬 좋을것 같다고 생각함.

decay_epoch = [32000,48000]
step_lr_scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=decay_epoch, gamma = 0.1)
#이거는 Learning rate decay를 표현하는 식이다.
#gamma가 0.1이니 한번 decay 될때마다 0.1씩 되는 것이고, 32000epoch에서 한번, 48000epoch에서 한번 작동한다.

writer = SummaryWriter(args.logdir)
#요약 작성 (자동으론 logs란 파일에 작성될 것)


Making model
The number of parameters of model is 464154
ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layers_2n): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16

대망의 Training 함수이다.

In [ ]:
def train(epoch, global_steps):
  net.train()

  train_loss = 0
  correct = 0
  total = 0

  for batch_idx, (inputs, targets) in enumerate(train_loader):
    #enumerate는 iterator임. 
    #앞서 train_loader를 정의할때 넣어준 batch_size만큼 for 문이 도는 것
    global_steps +=1
    step_lr_scheduler.step()
    #이 친구는 독립적을 움직이니 이렇게 추가해줘야해.

    inputs = inputs.to(device)
    targets = targets.to(device)
    outputs = net(inputs)
    loss = criterion(outputs, targets)

    optimizer.zero_grad()
    #역전파를 계산전에 변화도를 0으로 설정.
    #backward가 grad를 계산하게 되는데, autograd랑 충돌하니까 끈다는 느낌 (잔업이다)
    loss.backward()
    optimizer.step()
    #back propagation을 계산.

    train_loss += loss.item()
    _, predicted = outputs.max(1)
    #top 1 predict를 얻음
    total += targets.size(0)
    correct += predicted.eq(targets).sum().item()
    #predict랑 target이 같으면 correct를 올림. total을 그냥 올림.

  acc = 100 * correct/ total
  #정확도를 정의.
  print('train epoch : {} [{}/{}] | loss : {:.3f} | acc : {:.3f}'.format(epoch,batch_idx,len(train_loader), train_loss/(batch_idx+1),acc))
  #epoch마다 결과를 출력하는 것.

  writer.add_scalar('log/train error', 100-acc, global_steps)

  return global_steps



테스트 함수임.

In [ ]:
def test(epoch, best_acc, global_steps):
  net.eval()

  test_loss = 0
  correct = 0
  total = 0

  with torch.no_grad():
    #여기서 하는 일은 history tracking을 안하겠다는 것.
    #testing이니까 속도를 높여주려 그런게 아닐까?
    for batch_idx, (inputs, targets) in enumerate(test_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)
      outputs = net(inputs)
      loss = criterion(outputs, targets)

      test_loss += loss.item()
      _, predicted = outputs.max(1)
      total+= targets.size(0)
      correct += predicted.eq(targets).sum().item()

  acc = 100 * correct / total
  print('test epoch : {} [{}/{}] | loss : {:.3f} | acc : {:.3f}'.format(epoch,batch_idx,len(test_loader), test_loss/(batch_idx+1),acc))

  writer.add_scalar('log/train error', 100-acc, global_steps)
  #위의 train에서 backward()만 뺀거라 설명할게 크게 없다.


  if acc > best_acc:
    print('Saving model')
    state = {'net':net.state_dict(),'acc':acc, 'epoch':epoch,}
    
    if not os.path.isdir('save_model'):
      #저장 폴더가 없으면 만들기
      os.mkdir('save_model')
    torch.save(state,'./save_model/ckpt.pth')
    best_acc = acc
  #최고 기록인 모델 저장하기

  return best_acc


드디어 대망의 main함수. 근데 뭐 그러듯 별 볼거 없다.

In [ ]:
if __name__=='__main__':

  best_acc = 0
  epoch = 0
  global_steps = 0

  if args.resume is not None:
    #저장된게 있었으면, 그 모델로 test를 한다. (추가적인 train을 하진 않게 설정했네)
    test(epoch = 0, best_acc = 0)
  else:
    while True:
      epoch +=1
      global_steps = train(epoch, global_steps)
      best_acc = test(epoch, best_acc, global_steps)
      print('best test accuracy is ', best_acc)

      if global_steps >= 64000:
        #탈출조건
        break


KeyboardInterrupt: ignored